<a href="https://colab.research.google.com/github/Dandakandarpa/Tubes-prognet/blob/main/labeling_comment_youtube.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 75.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 112.5 MB/s eta 0:00:00


In [ ]:
!pip install googletrans==3.1.0a0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 7.4 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16352 sha256=63ff433eb1e0d69d5bc3bc9e566145f257efbceba0c450cf50641fdd97aea35c
  Stored in directory: /root/.cache/pip/wheels/50/5d/3c/8477d0af4ca2b8b1308812c09f1930863caeebc762fe265a95
Successfully built googletrans


In [ ]:
import pandas as pd

In [ ]:
import numpy as np
import re

In [ ]:
data_path = 'comment youtube preprocessing.csv'

In [ ]:
data = pd.read_csv(data_path)

In [ ]:
data.sample(5)

,Unnamed: 0,publishedAt,authorDisplayName,textDisplay,likeCount
1783,1783,2023-03-30T19:25:41Z,masberto1000,kang bakso yang filosofis,0
1445,1445,2023-03-31T04:15:52Z,Muhammad Muqbil Maulana,bang kamu main yang ori atau bajakan,1
385,385,2023-04-04T13:02:03Z,LeyLucy,hikmah yang bisa kita ambil dari game ini seku...,0
1640,1640,2023-03-30T22:44:07Z,rin,akhirnya game ini lagi,0
514,514,2023-04-02T15:37:49Z,Satriya Dhany,muka lu bang kayak,0


In [ ]:
print(f'shape: {data.shape}')

shape: (2612, 5)


In [ ]:
from googletrans.client import Translator
translator = Translator()

from transformers import pipeline
sentiment_classifier = pipeline('sentiment-analysis')

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Xformers is not installed correctly. If you want to use memorry_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [ ]:
def clean_tweet(tweet):
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ", tweet).split())

In [ ]:
data['label'] = data['textDisplay'].str.encode('ascii','ignore').apply(translator.translate, src='id', dest='en')
data['label'] = data['label'].apply(getattr, args=('text',))
data['label'] = data.apply(lambda x: clean_tweet(x['label']), axis=1)

In [ ]:
df = (
    data
    .assign(sentiment = lambda x: x['label'].apply(lambda s: sentiment_classifier(s)))
    .assign(
        label = lambda x: x['sentiment'].apply(lambda s: (s[0]['label'])),
        score = lambda x: x['sentiment'].apply(lambda s: (s[0]['score']))
    )
)

In [ ]:
df.sample(20)

,Unnamed: 0,publishedAt,authorDisplayName,textDisplay,likeCount,label,sentiment,score
2532,2532,2023-03-30T14:25:33Z,mixue hitam rasa coklat,halo bang windah,0,NEGATIVE,"[{'label': 'NEGATIVE', 'score': 0.732532799243...",0.732533
2244,2244,2023-03-30T14:44:25Z,Raiya Satriadjie22,awesome,0,POSITIVE,"[{'label': 'POSITIVE', 'score': 0.999836802482...",0.999837
893,893,2023-03-31T23:39:34Z,Horanghae,aku smk bang jujur gada yg sebrutal ini ngomon...,0,NEGATIVE,"[{'label': 'NEGATIVE', 'score': 0.989950835704...",0.989951
1189,1189,2023-03-31T10:11:48Z,Devgan Navy,aku jadi raja gelud abis nonton ini,0,POSITIVE,"[{'label': 'POSITIVE', 'score': 0.998404324054...",0.998404
407,407,2023-04-04T07:53:43Z,Damar jr,bang windah elus kucing tiga kali saat di seko...,1,NEGATIVE,"[{'label': 'NEGATIVE', 'score': 0.994482517242...",0.994483
2314,2314,2023-03-30T14:31:26Z,RaditMenggokil,enggaa bang aku anak smk,2,POSITIVE,"[{'label': 'POSITIVE', 'score': 0.999517321586...",0.999517
949,949,2023-03-31T20:12:03Z,Justin Nimrod Navarro,dulu pa nonton pre testnya ibu gua masi idup s...,0,NEGATIVE,"[{'label': 'NEGATIVE', 'score': 0.999012947082...",0.999013
455,455,2023-04-03T12:13:20Z,ikii!,game ini tu khusus karena ada kata kata yang s...,0,POSITIVE,"[{'label': 'POSITIVE', 'score': 0.987891674041...",0.987892
1858,1858,2023-03-30T17:54:58Z,Qeysya Sayyidina Bani,richard suaranya ganteng bgt,0,POSITIVE,"[{'label': 'POSITIVE', 'score': 0.999686241149...",0.999686
1336,1336,2023-03-31T06:38:36Z,kam Sholeh,awesome,1,POSITIVE,"[{'label': 'POSITIVE', 'score': 0.999836802482...",0.999837


In [ ]:
data_jadi = df.to_csv('label.csv', index=False)

In [ ]:
dt = pd.read_csv('label.csv')

In [ ]:
dt.head(5)

,Unnamed: 0,publishedAt,authorDisplayName,textDisplay,likeCount,label,sentiment,score
0,0,2023-03-30T22:04:06Z,MEMEXDLMAO,game ga mendidik bikin indonesia di pandang je...,2379,NEGATIVE,"[{'label': 'NEGATIVE', 'score': 0.998561680316...",0.998562
1,1,2023-04-02T22:25:35Z,RADENMIFTAHUL IKRAM,wkwkwk baru tau orang luar ngarti bahasa indo,0,NEGATIVE,"[{'label': 'NEGATIVE', 'score': 0.797197222709...",0.797197
2,2,2023-04-01T10:20:59Z,Fahmi,padahal ada youtube kidskenapa ngak pake itu h...,0,NEGATIVE,"[{'label': 'NEGATIVE', 'score': 0.995094537734...",0.995095
3,3,2023-04-01T10:01:13Z,raihangr~,apasi kamu yakali game harus persis dunia nyata,0,NEGATIVE,"[{'label': 'NEGATIVE', 'score': 0.722537219524...",0.722537
4,4,2023-04-01T10:01:02Z,Messong,lah ngaca napa banyak oknum orang indonesia ya...,0,NEGATIVE,"[{'label': 'NEGATIVE', 'score': 0.992120981216...",0.992121
